<h1> Non-Linear Regression For Classification </h1> <br>
<b>Example of how to implement an MLP with 1 hidden layer using numpy</b>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm

<h3> Loading the data </h3>
Lets load some "toy" data that we can use

In [ ]:
# you can load your data using this cell
npzfile = np.load("../data/toy_data_two_moon.npz") # toy_data.npz or toy_data_two_circles.npz

#The compressed Numpy file is split up into 4 parts
#Train inputs and target outputs
#Test inputs and target outputs
x_train = npzfile['arr_0']
x_test = npzfile['arr_1']
y_train = npzfile['arr_2']
y_test = npzfile['arr_3']

# remember that each row in x_train and X_test is a sample. so x_train[1,:] is the first training sample

<h3> Let's plot our data </h3>

In [ ]:
#Lets see what the data looks like
fig = plt.figure(figsize = (10, 5))
plt.subplot(121)
plt.scatter(x_train[:, 0], x_train[:, 1], marker='o', c=y_train[:,0], s=20, edgecolor='k')
plt.title("Train data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.subplot(122)
plt.scatter(x_test[:, 0], x_test[:, 1], marker='o', c=y_test[:,0], s=20, edgecolor='k')
plt.title("Test data")
plt.xlabel("X0")
plt.ylabel("X1")

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x) + 1e-4)

def relu(x):
    return (x > 0) * x

In [ ]:
class MLP():
    def __init__(self, input_size, output_size, hidden_size):
        self.theta0 = np.random.randn(input_size, hidden_size)/(input_size)
        self.bias0 = np.zeros((1, hidden_size))
        self.theta1 = np.random.randn(hidden_size, 1)/(hidden_size)
        self.bias1 = np.zeros((1, 1))

    def predict(self, x):
        h1 = np.matmul(x , self.theta0)
        h1_ = h1 + self.bias0
        h2 = relu(h1_)
        output = np.matmul(h2 , self.theta1) + self.bias1
        pred = (output >= 0.5).astype(int)
        return pred, (h1, h1_, h2, output)
    
    def compute_grad(self, x, y):
        _, layers = self.predict(x)
        h1, h1_, h2, output = layers

        dl_dtheta1 = np.matmul(h2.T , 2 * (output - y))/y.shape[0]
        dl_dbias1 = np.matmul(np.ones(output.shape).T, 2 * (output - y))/y.shape[0]

        dl_dh2 = np.matmul(2 * (output - y), self.theta1.T)
        
        # If using tanh
#         dl_dh1 = dl_dh2 * (1 - (tanh(h1) ** 2))

        # If using sigmoid
#         dl_dh1 = dl_dh2 * (sigmoid(h1)*(1 - sigmoid(h1)))

        # If using relu
        dl_dh1_ = dl_dh2 * (h1_ > 0)

        dl_dtheta0 = np.matmul(x.T , dl_dh1_)/y.shape[0]
        dl_dbias0 = np.matmul(np.ones(output.shape).T , dl_dh1_)/y.shape[0]

        return dl_dtheta0, dl_dbias0, dl_dtheta1, dl_dbias1
    
    def update_params(self, x, y, lr):
        dl_dtheta0, dl_dbias0, dl_dtheta1, dl_dbias1 = self.compute_grad(x, y)
        self.theta0 -= lr * dl_dtheta0 
        self.bias0 -= lr * dl_dbias0 
                              
        self.theta1 -= lr * dl_dtheta1
        self.bias1 -= lr * dl_dbias1
                              
    def compute_loss(self, x, y):
        _, layers = self.predict(x)
        _, _, _, output = layers
        return np.mean((output - y)**2)


In [ ]:
mlp = MLP(input_size=2, output_size=1, hidden_size=64)

In [ ]:
loss_log = []  # keep track of the loss values
acc = []  # keep track of the accuracy 

In [ ]:
lr = 0.05

# number of times we itterate over the dataset
max_epoch = 10000

for epoch in trange(max_epoch):
    y_test_hat, _ = mlp.predict(x_test)

    acc.append(float(sum(y_test_hat == y_test))/ float(len(y_test)))

    # call the compute_grad_loss that is implemented above to 
    # measure the loss and the gradient
    loss = mlp.compute_loss(x_train, y_train)
    mlp.update_params(x_train, y_train, lr)

    loss_log.append(loss)

print("Test Accuracy of linear model(GD): %.2f%% " %(acc[-1]*100))

In [ ]:
_ = plt.plot(acc)
_ = plt.title("Model accuracy per itteration")

In [ ]:
_ = plt.plot(loss_log)
_ = plt.title("Model loss per itteration")

In [ ]:
y_test_hat, _ = mlp.predict(x_test)
_ = plt.scatter(x_test[:, 0], x_test[:, 1], marker='o', c=y_test_hat[:,0], s=25, edgecolor='k')
_ = plt.title("Model Test Prediction")